In [3]:
from scripts.utils.utils import load_json
import pandas as pd
import numpy as np

In [19]:
networks = load_json('../data/test/networks/networks.json')
df = pd.read_parquet('/home/gezerli/Desktop/projects/reward-network-ii/data/test/preselection/preselection.parquet.gzip')
df_old=pd.read_parquet('/home/gezerli/Desktop/projects/reward_networks/data/exp1/preselection/preselection.parquet.gzip')
df_class = pd.read_parquet('/home/gezerli/Desktop/projects/reward-network-ii/data/test/derivative_classification/classification.parquet.gzip')

In [14]:
networks[4]

{'directed': True,
 'multigraph': False,
 'graph': {'directed': True},
 'nodes': [{'id': 0}, {'id': 5}, {'id': 4}, {'id': 1}, {'id': 2}, {'id': 3}],
 'links': [{'weight': 140, 'action': 1, 'source': 0, 'target': 5},
  {'weight': -20, 'action': 0, 'source': 0, 'target': 4},
  {'weight': -20, 'action': 1, 'source': 5, 'target': 2},
  {'weight': -100, 'action': 0, 'source': 5, 'target': 0},
  {'weight': 140, 'action': 1, 'source': 4, 'target': 1},
  {'weight': -20, 'action': 0, 'source': 4, 'target': 3},
  {'weight': 140, 'action': 1, 'source': 1, 'target': 0},
  {'weight': -100, 'action': 0, 'source': 1, 'target': 2},
  {'weight': 20, 'action': 0, 'source': 2, 'target': 5},
  {'weight': -100, 'action': 1, 'source': 2, 'target': 3},
  {'weight': 20, 'action': 0, 'source': 3, 'target': 5},
  {'weight': 20, 'action': 1, 'source': 3, 'target': 1}],
 'network_id': '10_4'}

In [17]:
df

,index,Network_ID,Starting_Node,Preselection
0,0,10_3,5,Just without outlier
1,1,10_5,5,Just without outlier
2,2,10_5,2,Just without outlier
3,3,10_2,0,Just without outlier
4,4,10_4,3,Just without outlier
5,5,10_1,3,Just without outlier
6,6,10_4,5,Just without outlier
7,7,10_5,1,Just without outlier
8,8,10_5,4,Just without outlier
9,9,10_4,0,Just without outlier


In [25]:
df_class

,Preselection,Network_ID,Starting_Node,Class
0,Just without outlier,10_1,1,Neutral
1,Just without outlier,10_1,2,Neutral
2,Just without outlier,10_1,3,Neutral
3,Just without outlier,10_1,4,Neutral
4,Just without outlier,10_1,5,Neutral
5,Just without outlier,10_2,0,Neutral
6,Just without outlier,10_2,5,Neutral
7,Just without outlier,10_3,0,Neutral
8,Just without outlier,10_3,1,Neutral
9,Just without outlier,10_3,5,Riskseeking Regretfull


In [23]:
df

,index,Network_ID,Starting_Node,Preselection
0,0,10_3,5,Just without outlier
1,1,10_5,5,Just without outlier
2,2,10_5,2,Just without outlier
3,3,10_2,0,Just without outlier
4,4,10_4,3,Just without outlier
5,5,10_1,3,Just without outlier
6,6,10_4,5,Just without outlier
7,7,10_5,1,Just without outlier
8,8,10_5,4,Just without outlier
9,9,10_4,0,Just without outlier
